This file takes in the input files, merges them, and cleans up the data while adding in buy or sell labels to help the RF models detect what's a good trade and what isn't

Todo: Comments (ChatGPT!)

Currently uses 7 stocks

# Merging data

In [2]:
### Adjust inputs if in Google Colab
# Process: Upload this notebook and any needed files to a Google Drive Folder named "Project_10" in the same folder structure.
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if not IN_COLAB:
    if ('full_added' in locals()) or ('full_added' in globals()):
        price_path = "Inputs/Downloaded/price_data_1991-2020" + test_added + full_added + ".csv"
        fund_path = "Inputs/Downloaded/Fundamentals_1991-2020" + test_added + full_added + ".dta"
        conversion_path = 'Inputs/Downloaded/gvkey_permno_conversion.dta'
    else:
        test_added = ''
        full_added = '_full'
        price_path = "../Inputs/Downloaded/price_data_full" + test_added + full_added + ".csv"
        fund_path = "../Inputs/Downloaded/Fundamentals_full" + test_added + full_added + ".dta"
        conversion_path = '../Inputs/Downloaded/gvkey_permno_conversion.dta'

else:
    # A few changes done for Google Colab
    from google.colab import drive
    # drive.mount('/content/drive')

    # # my_path = ""
    # gdrive_path = "/content/drive" + "/My Drive" + "/Project_10/" # THIS is your GDrive path
    # gpath_scripts = gdrive_path + "Background_Scripts/"
    # gpath_output = gdrive_path + "Outputs/"
    # gpath_inputs = gdrive_path + "Inputs/"

    # price_data = gpath_inputs + "price_data_1991" + full_added +  test_added + ".csv"
    # fund_data = gpath_inputs + "Fundamentals_1991" + full_added + test_added + ".dta"
    # conversion = gpath_inputs + "gvkey_permno_conversion.dta"
    # merge = gpath_inputs + "merged" + full_added + test_added + ".dta"

    # port = gpath_scripts + "portfolio_db.ipynb"
    # backtest_ex = gpath_scripts + "backtest_executor.ipynb"
    # backtest_stat = gpath_scripts + "backtest_statistician.ipynb"

In [ ]:
### Dataframe imports
try:
    price_df = price_data.copy()
    signal_df = fund_data.copy()
    gvkey_permno_conversion_df = pd.read_stata(conversion_path).drop(['index', 'fyearq', 'fqtr'], axis = 1)
except:
    price_df = pd.read_csv(price_path)
    signal_df = pd.read_stata(fund_path)
    gvkey_permno_conversion_df = pd.read_stata(conversion_path).drop(['index', 'fyearq', 'fqtr'], axis = 1)

In [ ]:
#############
# Constants #
#############
# Number of days between quarterly earnings announcement and when we can use data
# We can use data of up to 4 quarters ago, reported in the last 2 (we always use the most recent data)
# Accounting lag is for the time frame of the data, which we won't know until it's reported so we can't use it, which complicates our merging
# Report lag is for when the data is reported
min_accounting_lag = 1
max_accounting_lag = 361
min_report_date_lag = 1
max_report_date_lag = 181
min_lag_dt = np.timedelta64(min_accounting_lag,'D')
max_lag_dt = np.timedelta64(max_accounting_lag,'D')
min_date_lag_val = 1000000000*60*60*24
max_acc_lag_val = 361*min_date_lag_val
max_report_date_lag_val = 181*min_date_lag_val

# # Define thresholds for classification
default_buy_threshold = 0.02
default_sell_threshold = -0.02

# Currently based on days. small_ret = 1, big_ret = 12 if monthly data
small_ret = 21 # 1 month's worth of trading days
big_ret = 252 # 1 year's worth of trading days

# Minimum share price to open a new position
min_share_price = 1.0

################
# Data cleanup #
################
# Cols needed to cut down the columns later down the line
date_identifying_cols = ['permno', 'int_datadate', 'int_rdq']
price_id_cols = ['permno', 'int_date']

# Column renames, edits, merge dfs, drop important nulls
price_df = price_df.rename(columns = {'PERMNO':'permno'})
price_df['RET'] = pd.to_numeric(price_df['RET'], errors='coerce')
price_df = price_df[price_df['RET'].notna()]
price_df.loc[:,'date'] = pd.to_datetime(price_df.loc[:,'date'], format ="%Y-%m-%d")
print('merging signal and gvkey')
comb_df = signal_df.merge(gvkey_permno_conversion_df, on=['gvkey','datadate'])
comb_df.drop('gvkey', axis = 1, inplace = True)
comb_df['permno'] = comb_df['permno'].astype(np.int64)

del gvkey_permno_conversion_df

# This gets rid of datapoints where the stock doesn't have a reporting date for accounting info
comb_df = comb_df[pd.notnull(comb_df['rdq'])] # Should probably do this for other datapoints

#####################
# Setup for merging #
#####################
# Make it easier to check dates by changing to numbers
price_df['int_date'] = price_df['date'].apply(lambda x: x.value)
comb_df['int_datadate'] = comb_df['datadate'].apply(lambda x: x.value)
comb_df['int_rdq'] = comb_df['rdq'].apply(lambda x: x.value)

# Some constants used later
unique_permnos = price_df['permno'].unique()
# good_indices = []
# all_s = pd.Series()

price_df = price_df.sort_values(by=['permno', 'int_date'])
comb_df = comb_df.sort_values(by=['permno', 'int_rdq', 'int_datadate'])

In [5]:
def new_price_df(price_df):

    permno_to_date_dict = {}
    # total_time0 = 0
    # total_time1 = 0
    # total_time2 = 0
    # total_time3 = 0
    # total_time4 = 0
    # t0=time.time()

    grouped = price_df.groupby('permno')

    for p in unique_permnos:
        this_permno_dict = {}
        # t1 = time.time()
        curr_price_df = grouped.get_group(p)
        current_trading_days = curr_price_df['int_date'].unique()

        curr_signal_df = comb_df[comb_df['permno'] == p]
        if curr_price_df.empty or curr_signal_df.empty:
            continue
        # t2 = time.time()
        # total_time0 += (t2-t1)
        
        int_rdq_list = list(curr_signal_df.loc[:,'int_rdq'])
        int_datadate_list = list(curr_signal_df.loc[:,'int_datadate'])
        
        signal_pointer = 0
        for date in current_trading_days:
            if signal_pointer >= len(curr_signal_df):
                break
            # t11 = time.time()
            rdq_date = int_rdq_list[signal_pointer]
            # t111 = time.time()
            # total_time1 += (t111-t11)
            if date <= rdq_date:
                continue
            doPrint = True

            # t22 = time.time()
            curr_condition = (date>rdq_date) and ((date-rdq_date)<max_report_date_lag_val) and ((date-int_datadate_list[signal_pointer])<max_acc_lag_val)
            # t222 = time.time()
            # total_time2 += (t222-t22)
            if signal_pointer + 1 >= len(curr_signal_df):
                next_condition = False
            else:
                # t33 = time.time()
                next_rdq_date = int_rdq_list[signal_pointer+1]
                # t333 = time.time()
                # total_time3 += (t333-t33)

                # t44 = time.time()
                next_condition = (date>next_rdq_date) and ((date-next_rdq_date)<max_report_date_lag_val) and ((date-int_datadate_list[signal_pointer+1])<max_acc_lag_val)
                # t444 = time.time()
                # total_time4 += (t444-t44)
            while not curr_condition or (curr_condition and next_condition):
                if signal_pointer+1 >= len(curr_signal_df):
                    doPrint = False
                    break
                signal_pointer += 1
                curr_condition = (date>rdq_date) and (date-rdq_date<max_report_date_lag_val) and (date-int_datadate_list[signal_pointer]<max_acc_lag_val)
                if signal_pointer + 1 >= len(curr_signal_df):
                    next_condition = False
                else:
                    next_rdq_date = int_rdq_list[signal_pointer+1]
                    next_condition = (date>next_rdq_date) and ((date-next_rdq_date)<max_report_date_lag_val) and ((date-int_datadate_list[signal_pointer+1])<max_acc_lag_val)
            try:
                if not doPrint:
                    continue
                this_permno_dict[date] = rdq_date
                # print('added')
            except:
                a = 0

        if this_permno_dict != {}:
            permno_to_date_dict[p] = this_permno_dict
    # print(total_time0)
    # print(total_time1)
    # print(total_time2)
    # print(total_time3)
    # print(total_time4)
    # print(time.time()-t0)

    price_df['int_rdq'] = price_df.apply(
        lambda row: permno_to_date_dict.get(row['permno'], {}).get(row['int_date'], None),
        axis=1
    )
    gc.collect()
    return price_df

price_df = new_price_df(price_df)

In [ ]:
gc.collect()
print('p', price_df[['permno', 'int_rdq']].head())
print('col', comb_df.columns)
print('c', comb_df[['permno', 'int_rdq']].head())
print('u1', price_df['int_rdq'].unique())
print('u2', comb_df['int_rdq'].unique())
merged_df = pd.merge(price_df, comb_df, on=['permno', 'int_rdq'], how='left')
del price_df
del comb_df
new_m = merged_df[merged_df['int_rdq'].notna()]
del merged_df
gc.collect()

## Below signals are added

In [ ]:
try:
    %run Background_Scripts/signal_info.py
except:
    %run ../Background_Scripts/signal_info.py

gc.collect()
new_m = add_signals_to_df(new_m, small_ret, big_ret, default_buy_threshold, default_sell_threshold)
new_m['permno'] = new_m['permno'].astype(str)
gc.collect()

In [8]:
# new_m.to_csv("../Inputs/Created/merged_full.csv")
# gc.collect()

In [ ]:
# Saving the file:
# new_m['permno'] = new_m['permno'].astype(str)

if IN_COLAB:
    new_m.to_stata(merge, convert_dates={'date': 'tc', 'datadate': 'tc', 'rdq': 'tc'})
else:
    # full_added = '_full'
    try:
        new_m.to_stata("Inputs/Created/merged" + full_added + test_added + ".dta", convert_dates={'date': 'tc', 'datadate': 'tc', 'rdq': 'tc'})
    except:
        new_m.to_stata("../Inputs/Created/merged" + full_added + test_added + ".dta", convert_dates={'date': 'tc', 'datadate': 'tc', 'rdq': 'tc'})
gc.collect()